In [93]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [94]:
df = pd.read_csv("../UH_2023/UH_2023_TRAIN.txt", sep = "|")

In [95]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [96]:
df

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
0,14,76953,515,4,660,26,2,0,1,0.0000,22215.0
1,14,84318,515,4,660,26,2,0,1,0.0000,22215.0
2,14,85579,340,4,520,32,2,0,1,0.0000,20978.0
3,14,69671,340,4,520,32,2,0,1,0.0000,40722.0
4,14,14001,852,14,NaN,81,1,0,1,0.0000,14126.0
...,...,...,...,...,...,...,...,...,...,...,...
9596,22,37461,239,6,700,52,2,0,1,3.6800,NaN
9597,22,58769,239,6,700,32,2,0,1,4.2500,NaN
9598,22,58769,239,6,700,59,2,0,1,4.0700,NaN
9599,22,88928,239,6,700,40,2,0,1,4.5727,NaN


In [97]:
import category_encoders as ce
encoder = ce.OrdinalEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "COLOR", "TIPO", "MODO"])
df = encoder.fit_transform(df)

In [99]:
df

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
0,14,1,1,1,660,1,1,1,1,0.0000,22215.0
1,14,2,1,1,660,1,1,1,1,0.0000,22215.0
2,14,3,2,1,520,2,1,1,1,0.0000,20978.0
3,14,4,2,1,520,2,1,1,1,0.0000,40722.0
4,14,5,3,2,NaN,3,2,1,1,0.0000,14126.0
...,...,...,...,...,...,...,...,...,...,...,...
9596,22,909,104,18,700,8,1,1,1,3.6800,NaN
9597,22,911,104,18,700,2,1,1,1,4.2500,NaN
9598,22,911,104,18,700,5,1,1,1,4.0700,NaN
9599,22,912,104,18,700,6,1,1,1,4.5727,NaN


In [100]:
encoder.inverse_transform(df)

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
0,14,76953,515,4,660,26,2,0,1,0.0000,22215.0
1,14,84318,515,4,660,26,2,0,1,0.0000,22215.0
2,14,85579,340,4,520,32,2,0,1,0.0000,20978.0
3,14,69671,340,4,520,32,2,0,1,0.0000,40722.0
4,14,14001,852,14,NaN,81,1,0,1,0.0000,14126.0
...,...,...,...,...,...,...,...,...,...,...,...
9596,22,37461,239,6,700,52,2,0,1,3.6800,NaN
9597,22,58769,239,6,700,32,2,0,1,4.2500,NaN
9598,22,58769,239,6,700,59,2,0,1,4.0700,NaN
9599,22,88928,239,6,700,40,2,0,1,4.5727,NaN


In [5]:
df_alt = df.drop(columns = ["SUPERFICIE", "PRODUCCION"])
X_df_alt = df_alt.drop(columns = ["ALTITUD", "CAMPAÑA"])
y_df_alt = df_alt["ALTITUD"]

X_df_alt_train = X_df_alt[df["ALTITUD"].isna() == False]
y_df_alt_train = y_df_alt[df["ALTITUD"].isna() == False]

X_to_predict = X_df_alt[df["ALTITUD"].isna() == True]

In [6]:
y_df_alt_train

0           660
1           660
2           520
3           520
12      650-660
         ...   
9596        700
9597        700
9598        700
9599        700
9600        700
Name: ALTITUD, Length: 9547, dtype: object

In [7]:
X_df_alt_train_n390 = X_df_alt_train[y_df_alt_train != "390"]
y_df_alt_train_n390 = y_df_alt_train[y_df_alt_train != "390"]

In [22]:
X_df_alt_train[y_df_alt_train == "390"]

,ID_FINCA,ID_ZONA,ID_ESTACION,VARIEDAD,MODO,TIPO,COLOR
1139,906,102,17,9,2,1,2


In [8]:
y_df_alt_train_n390_enc = pd.Series(LabelEncoder().fit_transform(y_df_alt_train_n390))

In [9]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

def grid(X,y):
    gsc = GridSearchCV(
                estimator=xgb.XGBClassifier(),
                param_grid={"n_estimators": [300, 500, 1000],
                            "learning_rate": [0.15],
                            "max_depth": [40, 60, 100],
                            "min_child_weight": [1, 2],
                            "gamma":[0.0],
                            "colsample_bytree":[0.4],},
                cv=2, scoring=["r2", "neg_root_mean_squared_error"], refit = "r2", verbose=0, n_jobs=-1)
    gsc.fit(X,y)
    return gsc

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

def grid2(X,y):
    gsc = GridSearchCV(
                estimator=xgb.XGBClassifier(),
                param_grid={"n_estimators": [300],
                            "learning_rate": [0.15],
                            "max_depth": [40],
                            "min_child_weight": [1],
                            "gamma":[0.0],
                            "colsample_bytree":[0.4],},
                cv=2, scoring=["r2", "neg_root_mean_squared_error"], refit = "r2", verbose=0, n_jobs=-1)
    gsc.fit(X,y)
    return gsc

In [10]:
alt_grid = grid2(X_df_alt_train_n390, y_df_alt_train_n390_enc)

In [11]:
params_model = alt_grid.best_params_

In [23]:
print(params_model)

{'colsample_bytree': 0.4, 'gamma': 0.0, 'learning_rate': 0.15, 'max_depth': 40, 'min_child_weight': 1, 'n_estimators': 300}


In [12]:
label_enc = LabelEncoder().fit(y_df_alt_train)
y_df_alt_train_enc = pd.Series(label_enc.transform(y_df_alt_train))

In [13]:
model_alt_grid = xgb.XGBClassifier(**params_model)
model_alt_grid.fit(X_df_alt_train,y_df_alt_train_enc)
predicted = model_alt_grid.predict(X_to_predict)

In [14]:
df2 = pd.read_csv("../UH_2023/UH_2023_TRAIN.txt", sep = "|")
df2 = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [15]:
predicted2 = pd.Series(label_enc.inverse_transform(predicted))
predicted2.index = df2[df2["ALTITUD"].isna() == True].index

In [16]:
df2.fillna({"ALTITUD": predicted2}, inplace=True)

In [17]:
df2

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
0,14,1,1,1,660,1,1,1,1,0.0000,22215.0
1,14,2,1,1,660,1,1,1,1,0.0000,22215.0
2,14,3,2,1,520,2,1,1,1,0.0000,20978.0
3,14,4,2,1,520,2,1,1,1,0.0000,40722.0
4,14,5,3,2,650-660,3,2,1,1,0.0000,14126.0
...,...,...,...,...,...,...,...,...,...,...,...
9596,22,909,104,18,700,8,1,1,1,3.6800,NaN
9597,22,911,104,18,700,2,1,1,1,4.2500,NaN
9598,22,911,104,18,700,5,1,1,1,4.0700,NaN
9599,22,912,104,18,700,6,1,1,1,4.5727,NaN


In [18]:
def fun(x, mode): 
    altitud = str(x).split('-')
    if altitud[0] == "nan":
        if mode == "nan_1":
            return -1
        return np.nan
    if mode == "nan_1":
            return x
    if mode == "min":
        return int(altitud[0])
    if mode == "diff":
        if len(altitud) == 1:
            return 0
        else:
            return int(altitud[1])-int(altitud[0])

df2["ALTITUD_MIN"] = df2["ALTITUD"].apply(fun, args=("min",))
df2["ALTITUD_DIF"] = df2["ALTITUD"].apply(fun, args=("diff",))
df2["ALTITUD"] = df2["ALTITUD"].apply(fun, args=("nan_1",))

In [19]:
df2.loc[df["MODO"]=="1", "MODO"]  = "0"
df2.loc[df["MODO"]=="2", "MODO"]  = "1"

In [20]:
df2

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
0,14,1,1,1,660,1,1,1,1,0.0000,22215.0,660,0
1,14,2,1,1,660,1,1,1,1,0.0000,22215.0,660,0
2,14,3,2,1,520,2,1,1,1,0.0000,20978.0,520,0
3,14,4,2,1,520,2,1,1,1,0.0000,40722.0,520,0
4,14,5,3,2,650-660,3,2,1,1,0.0000,14126.0,650,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9596,22,909,104,18,700,8,1,1,1,3.6800,NaN,700,0
9597,22,911,104,18,700,2,1,1,1,4.2500,NaN,700,0
9598,22,911,104,18,700,5,1,1,1,4.0700,NaN,700,0
9599,22,912,104,18,700,6,1,1,1,4.5727,NaN,700,0


In [21]:
df_new = df2

### Dataset 1 - Tot

In [291]:
df_new.to_csv("../DATASETS_TRACTATS/TOT_altitud.csv", index=False)

### Dataset 2 - Tot menys 14-15

In [292]:
df_new2 = df_new.copy()
rows_to_drop = ["14", "15"]
df_new2 = df_new2[df_new2["CAMPAÑA"].isin(rows_to_drop) != True]

In [293]:
df_new2.to_csv("../DATASETS_TRACTATS/TOT_menys14_15_altitud.csv", index=False)

### Dataset 3 - Eliminant files de 14-15 on ID_FINCA_ZONA i VARIEDAD no hi són en el 22

In [294]:
df_new3 = df_new.copy()
df_new3["ID_FINCA_ZONA"] = df_new3["ID_FINCA"] + "_"+ df_new3["ID_ZONA"]

In [295]:
rows_not_to_drop_id_finca_zona = df_new3[df_new3["CAMPAÑA"] == "22"]["ID_FINCA_ZONA"].unique()
rows_not_to_drop_varietat = df_new3[df_new3["CAMPAÑA"] == "22"]["VARIEDAD"].unique
years_to_look = ["14", "15"]

In [296]:
df_new3 = df_new3[(df_new3["CAMPAÑA"].isin(years_to_look) == False)|
                  ((df_new3["CAMPAÑA"].isin(years_to_look) == True)&
                  (df_new3["ID_FINCA_ZONA"].isin(rows_not_to_drop_id_finca_zona) == True)&
                   (df_new3["ID_FINCA_ZONA"].isin(rows_not_to_drop_id_finca_zona) == True))]

In [297]:
df_new3 = df_new3.drop(columns= "ID_FINCA_ZONA")

In [298]:
df_new3.to_csv("../DATASETS_TRACTATS/TOT_fzv_1415_altitud.csv", index=False)

### Dataset 4 - Eliminant files de 14-21 on ID_FINCA_ZONA no hi són en el 22

In [299]:
df_new4 = df_new.copy()
df_new4["ID_FINCA_ZONA"] = df_new4["ID_FINCA"] + "_"+ df_new4["ID_ZONA"]

In [300]:
rows_not_to_drop_id_finca_zona = df_new4[df_new4["CAMPAÑA"] == "22"]["ID_FINCA_ZONA"].unique()
rows_not_to_drop_varietat = df_new4[df_new4["CAMPAÑA"] == "22"]["VARIEDAD"].unique
years_to_look = ["14", "15", "16", "17", "18", "19", "20", "21"]
df_new4 = df_new4[(df_new4["CAMPAÑA"].isin(years_to_look) == False)|
                  ((df_new4["CAMPAÑA"].isin(years_to_look) == True)&
                  (df_new4["ID_FINCA_ZONA"].isin(rows_not_to_drop_id_finca_zona) == True)&
                   (df_new4["ID_FINCA_ZONA"].isin(rows_not_to_drop_id_finca_zona) == True))]

In [301]:
df_new4 = df_new4.drop(columns= "ID_FINCA_ZONA")

In [302]:
df_new4.to_csv("../DATASETS_TRACTATS/TOT_fzv_1421_altitud.csv", index=False)

### Dataset 5 - Eliminant files de 14-21 on ID_FINCA_ZONA no hi són en el 22

In [42]:
df_new5 = df_new.copy()

In [25]:
df_16_21 = df_new[df_new["CAMPAÑA"].isin(["14", "15", "22"]) == False]
df_22 = df_new[df_new["CAMPAÑA"].isin(["22"]) == True]

In [39]:
id_finca = set(df_22["ID_FINCA"]) - set(df_16_21["ID_FINCA"])
id_zona = set(df_22["ID_ZONA"]) - set(df_16_21["ID_ZONA"])
id_estacion = set(df_22["ID_ESTACION"]) - set(df_16_21["ID_ESTACION"])
id_color = set(df_22["COLOR"]) - set(df_16_21["COLOR"])
id_tipo = set(df_22["TIPO"]) - set(df_16_21["TIPO"])
id_modo = set(df_22["MODO"]) - set(df_16_21["MODO"])

In [47]:
df_new5[df_new5["ID_ZONA"].isin(id_zona)==True]

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
9565,22,1227,125,15,440,7,2,1,2,3.1000,NaN,440,0
9566,22,1227,125,15,440,4,2,1,1,1.8300,NaN,440,0
9567,22,1227,125,15,440,5,2,1,1,5.8907,NaN,440,0


In [48]:
df_new5[df_new5["ID_FINCA"].isin(id_finca)==True]

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
261,14,193,32,9,605-615,4,2,1,1,0.0000,3960.0,605,10
8539,22,1194,7,2,650-660,7,1,1,2,0.0189,NaN,650,10
8540,22,1194,7,2,650-660,8,1,1,1,0.1326,NaN,650,10
8541,22,1195,7,2,650-660,7,1,1,2,0.4600,NaN,650,10
8542,22,1196,7,2,650-660,2,1,1,1,3.4711,NaN,650,10
8543,22,1196,7,2,650-660,6,1,1,1,2.6300,NaN,650,10
8567,22,1197,8,2,650-660,4,1,1,1,0.0342,NaN,650,10
8568,22,1198,8,2,650-660,4,1,1,1,0.0969,NaN,650,10
8569,22,1199,8,2,650-660,4,1,1,1,0.0869,NaN,650,10
8570,22,1200,8,2,650-660,5,1,1,1,2.7200,NaN,650,10


In [54]:
df_new5[df_new5["SUPERFICIE"] == 2.2643]

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
9583,22,1231,101,16,450-480,5,1,1,1,2.2643,NaN,450,30


In [59]:
df_new5[df_new5["SUPERFICIE"] == 7.5143]

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
7673,21,146,29,9,625-670,5,2,1,1,7.5143,11710.0,625,45
8746,22,146,29,9,625-670,5,2,1,1,7.5143,NaN,625,45


In [86]:
dd = encoder.inverse_transform(df_new5.drop(columns = ["ALTITUD_MIN", "ALTITUD_DIF"]))

In [101]:
df_new5

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF
0,14,1,1,1,660,1,1,1,1,0.0000,22215.0,660,0
1,14,2,1,1,660,1,1,1,1,0.0000,22215.0,660,0
2,14,3,2,1,520,2,1,1,1,0.0000,20978.0,520,0
3,14,4,2,1,520,2,1,1,1,0.0000,40722.0,520,0
4,14,5,3,2,650-660,3,2,1,1,0.0000,14126.0,650,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9596,22,909,104,18,700,8,1,1,1,3.6800,NaN,700,0
9597,22,911,104,18,700,2,1,1,1,4.2500,NaN,700,0
9598,22,911,104,18,700,5,1,1,1,4.0700,NaN,700,0
9599,22,912,104,18,700,6,1,1,1,4.5727,NaN,700,0


In [102]:
encoder.inverse_transform(df_new5.drop(columns = ["ALTITUD_MIN", "ALTITUD_DIF"]))

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
0,14,NaN,NaN,NaN,660,NaN,NaN,NaN,NaN,0.0000,22215.0
1,14,NaN,NaN,NaN,660,NaN,NaN,NaN,NaN,0.0000,22215.0
2,14,NaN,NaN,NaN,520,NaN,NaN,NaN,NaN,0.0000,20978.0
3,14,NaN,NaN,NaN,520,NaN,NaN,NaN,NaN,0.0000,40722.0
4,14,NaN,NaN,NaN,650-660,NaN,NaN,NaN,NaN,0.0000,14126.0
...,...,...,...,...,...,...,...,...,...,...,...
9596,22,NaN,NaN,NaN,700,NaN,NaN,NaN,NaN,3.6800,NaN
9597,22,NaN,NaN,NaN,700,NaN,NaN,NaN,NaN,4.2500,NaN
9598,22,NaN,NaN,NaN,700,NaN,NaN,NaN,NaN,4.0700,NaN
9599,22,NaN,NaN,NaN,700,NaN,NaN,NaN,NaN,4.5727,NaN
